In [4]:
print("All OK")

All OK


In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [9]:
chat_llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c53e1-372e-7e01-8046-4a1c91c3d081-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})

In [10]:
from typing_extensions import Annotated,TypedDict
import operator

In [11]:
from langchain_core.messages import AnyMessage,HumanMessage,AIMessage

In [12]:
class GraphState(TypedDict):
    message:Annotated[list[AnyMessage],operator.add]